## NBA Players
1. **Title** - Visualizing NBA Players on Latent Space

2. **Features**
 - Player_Info
     - DISPLAY_FIRST_LAST
     - BIRTHDATE
     - HEIGHT
     - WEIGHT
     - SCHOOL
     - POSITION
     - JERSEY
 - Career
     - Teams Played
     - Points Per Game
     - Assists Per Game
     - 3 Points Per Game
     - Steals Per Game
     - NBA Titles
     - MVP Titles
 
3. **Ideas**
 - Categorical Embeddings
 - Graphs
 - UMAP for dim reduction
 - Altair interactive for visualization

In [1]:
#!pip install -U pandas

In [2]:
from tqdm import tqdm_notebook
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats
from nba_api.stats.static import players
from nba_api.stats.static import teams
import pandas as pd
pd.__version__

'1.0.0'

In [3]:
# Find players by full name.
players.find_players_by_full_name('michael jordan')

[{'first_name': 'Michael',
  'full_name': 'Michael Jordan',
  'id': 893,
  'is_active': False,
  'last_name': 'Jordan'}]

In [4]:
# Find teams by full name.
teams.find_teams_by_full_name('mav')

[{'abbreviation': 'DAL',
  'city': 'Dallas',
  'full_name': 'Dallas Mavericks',
  'id': 1610612742,
  'nickname': 'Mavericks',
  'state': 'Texas',
  'year_founded': 1980}]

In [5]:
from nba_api.stats.endpoints import commonplayerinfo

jordan_id = 893
player_info = commonplayerinfo.CommonPlayerInfo(player_id = jordan_id, timeout=100)

KeyboardInterrupt: 

In [ ]:
player_info.available_seasons.get_json()

In [ ]:
player_info.common_player_info.get_data_frame()['JERSEY']

In [ ]:
def return_player_info(id):
    player_info = commonplayerinfo.CommonPlayerInfo(player_id = id, timeout=100)
    player_info_df = player_info.common_player_info.get_data_frame()
    data ={
        "DISPLAY_FIRST_LAST": player_info_df['DISPLAY_FIRST_LAST'].item(),
        "BIRTHDATE": player_info_df['BIRTHDATE'].item(),
        "HEIGHT": player_info_df['HEIGHT'].item(),
        "WEIGHT": player_info_df['WEIGHT'].item(),
        "SCHOOL": player_info_df['SCHOOL'].item(),
        "POSITION": player_info_df['POSITION'].item(),
        "JERSEY": player_info_df['JERSEY'].item(),
    }
    return data

In [ ]:
return_player_info(893)

## List PLayers

In [ ]:
from nba_api.stats.static import players

In [ ]:
active = players.get_active_players()

In [ ]:
inactive = players.get_inactive_players()

In [ ]:
active_ids = [x['id'] for x in active]
print(len(active_ids))

In [ ]:
893 in active_ids

In [ ]:
inactive_ids = [x['id'] for x in inactive]
print(len(inactive_ids))

In [ ]:
return_player_info(893)

In [ ]:
all_player_df = pd.DataFrame(columns= {'BIRTHDATE',
                                       'DISPLAY_FIRST_LAST',
                                       'HEIGHT',
                                       'WEIGHT',
                                       'JERSEY',
                                       'POSITION',
                                       'SCHOOL'
                                      })

In [ ]:
for i in tqdm_notebook(active_ids):
    all_player_df.append(return_player_info(i), ignore_index = True)

In [ ]:
all_player_df

## Career

In [ ]:
career = playercareerstats.PlayerCareerStats(player_id='893')
career.get_data_frames()[0]

## Sources
1. https://github.com/swar/nba_api/tree/master/docs/examples  
2. https://towardsdatascience.com/link-prediction-with-neo4j-part-2-predicting-co-authors-using-scikit-learn-78b42356b44c  
3. https://medium.com/@davidheffernan_99410/an-introduction-to-using-categorical-embeddings-ee686ed7e7f9  
4. https://medium.com/@george.drakos62/decoded-entity-embeddings-of-categorical-variables-in-neural-networks-1d2468311635  
5. https://pypi.org/project/entity-embeddings-categorical/  